In [1]:
import carla
import random

In [ ]:
# Подключение к симулятору python клиентом
client = carla.Client('localhost', 2000)
client.set_timeout(2.0)

In [3]:
client.load_world('Town03')

In [3]:
# Once we have a client we can retrieve the world that is currently
# running.
world = client.get_world()

# The world contains the list blueprints that we can use for adding new
# actors into the simulation.
blueprint_library = world.get_blueprint_library()

# Get the blueprint library and filter for the vehicle blueprints
vehicle_blueprints = world.get_blueprint_library().filter('*vehicle*')
# Get the map's spawn points
spawn_points = world.get_map().get_spawn_points()

# Basic tutorial

In [7]:
actor_list = []

In [ ]:
# Now let's filter all the blueprints of type 'vehicle' and choose one
# at random.
bp = random.choice(blueprint_library.filter('vehicle'))

# A blueprint contains the list of attributes that define a vehicle's
# instance, we can read them and modify some of them. For instance,
# let's randomize its color.
if bp.has_attribute('color'):
    color = random.choice(bp.get_attribute('color').recommended_values)
    bp.set_attribute('color', color)

In [5]:
# Now we need to give an initial transform to the vehicle. We choose a
# random transform from the list of recommended spawn points of the map.
transform = random.choice(world.get_map().get_spawn_points())

In [6]:
# So let's tell the world to spawn the vehicle.
vehicle = world.spawn_actor(bp, transform)

In [8]:
# It is important to note that the actors we create won't be destroyed
# unless we call their "destroy" function. If we fail to call "destroy"
# they will stay in the simulation even after we quit the Python script.
# For that reason, we are storing all the actors we create so we can
# destroy them afterwards.
actor_list.append(vehicle)
print('created %s' % vehicle.type_id)

created vehicle.diamondback.century


In [9]:
# Let's put the vehicle to drive around.
vehicle.set_autopilot(True)

In [10]:
# Let's add now a "depth" camera attached to the vehicle. Note that the
# transform we give here is now relative to the vehicle.
camera_bp = blueprint_library.find('sensor.camera.depth')
camera_transform = carla.Transform(carla.Location(x=1.5, z=2.4))
camera = world.spawn_actor(camera_bp, camera_transform, attach_to=vehicle)
actor_list.append(camera)
print('created %s' % camera.type_id)

created sensor.camera.depth


In [18]:
# Oh wait, I don't like the location we gave to the vehicle, I'm going
# to move it a bit forward.
location = vehicle.get_location()
location.x += 40
vehicle.set_location(location)
print('moved vehicle to %s' % location)

moved vehicle to Location(x=-116.591125, y=39.366714, z=-2.067356)


In [17]:
# Now we register the function that will be called each time the sensor
# receives an image. In this example we are saving the image to disk
# converting the pixels to gray-scale.
cc = carla.ColorConverter.LogarithmicDepth
camera.listen(lambda image: image.save_to_disk('_out/%06d.png' % image.frame, cc))

In [19]:
camera.destroy()

True

In [20]:
vehicle.destroy()

True

# Playground

In [4]:
# Spawn 50 vehicles randomly distributed throughout the map 
# for each spawn point, we choose a random vehicle from the blueprint library
for i in range(0,50):
    world.try_spawn_actor(random.choice(vehicle_blueprints), random.choice(spawn_points))

In [5]:
for vehicle in world.get_actors().filter('*vehicle*'):
    vehicle.set_autopilot(True)

In [6]:
ego_bp = world.get_blueprint_library().find('vehicle.seat.leon')

ego_bp.set_attribute('role_name', 'hero')

ego_vehicle = world.spawn_actor(ego_bp, random.choice(spawn_points))

In [ ]:
# Создание сенсора "камера"
camera_init_trans = carla.Transform(carla.Location(z=1.5))
camera_bp = world.get_blueprint_library().find('sensor.camera.rgb')
camera = world.spawn_actor(camera_bp, camera_init_trans, attach_to=ego_vehicle)

In [8]:
import cv2
import numpy as np

def show_image(image):
    # Преобразуем CARLA image в массив numpy
    array = np.frombuffer(image.raw_data, dtype=np.uint8)
    array = array.reshape((image.height, image.width, 4))  # BGRA формат

    # Избавляемся от альфа-канала (если не нужен)
    frame = array[:, :, :3]

    # Преобразуем BGRA -> RGB, если нужно (OpenCV по умолчанию в BGR)
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Показываем изображение
    cv2.imshow("CARLA Camera", frame)
    
    # Обновляем окно; 1 мс ожидания клавиши (нужно для обработки GUI)
    if cv2.waitKey(1) == 27:  # Нажми Esc чтобы выйти
        image.stop()


In [9]:
# Пример подключения:
camera.listen(show_image)

In [ ]:
# Управление: газ, поворот, тормоз
control = carla.VehicleControl()

In [45]:
control.steer = +0.00     # Поворот от -1.0 до 1.0
ego_vehicle.apply_control(control)

In [46]:
control.throttle = 0.5  # Газ от 0.0 до 1.0
control.brake = 0.0
ego_vehicle.apply_control(control)

In [47]:
# Остановим
control.throttle = 0.0
control.brake = 1.0
ego_vehicle.apply_control(control)